In [11]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import math
import requests
tqdm.pandas()

C:\Users\Javi\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [12]:
paper_df = pd.read_csv('./paper_dataset.csv')
movie_df = pd.read_csv('./preprocessed_movie_dataset.csv')

In [13]:
char_str_cols = paper_df.columns.to_list()[1:-1]

movie_df['Meaningfulness'] = 0.0

for col in char_str_cols[1:]:
    
    movie_df[col] = 0

In [23]:
movie_list = movie_df.iloc[:, 2].tolist()
years = movie_df.iloc[:, 4].tolist()
orig_movie_list = movie_df.iloc[:, 3].tolist()

for i, movie in enumerate(movie_list):
    movie_list[i] = movie.lower().strip().replace(' ', '')

for i, movie in enumerate(orig_movie_list):
    orig_movie_list[i] = movie.lower().strip().replace(' ', '')

##### It's impossible to match all of the movie due to the different formatting. If there isn't a perfect match, there is no point in adding the character strength and start with wrong scores, therefore the missing matches must be added manually

In [29]:
movie_d = "kk (2000)"
c = movie_d.split('(')[0].strip().lower().strip()
year = movie_d[movie_d.find('(')+1:movie_d.find(')')]

indices = [i for i, x in enumerate(movie_list) if x == c]

for idx in indices:
    if years[idx] == year:
        print('yes')
    else:
        print('no')

0


In [34]:
unmatched_movies = []
movie_columns = movie_df.columns.to_list()

for idx, row in tqdm(paper_df.iterrows(), total=paper_df.shape[0]):
    
    current_movie = row['Unnamed: 0'].split('(')[0].strip().lower().replace(' ', '')
    year = row['Unnamed: 0'][row['Unnamed: 0'].find("(")+1:row['Unnamed: 0'].find(")")]
    
    # find all ocurrences of the movie title in the list (there can be several movies with same name)
    indices = [i for i, movie in enumerate(movie_list) if movie == current_movie]

    # if not in the movie_list, check for the original name list
    if len(indices) == 0:
        
        indices = [i for i, movie in enumerate(orig_movie_list) if movie == current_movie]
        
        if len(indices) == 0:
            
            #print(f"Movie {current_movie} ({year}) was not found")
            to_print = []
            
            for col in char_str_cols[1:]:
                if row[col] > 0:
                    to_print.append(col)
                    
            unmatched_movies.append((row['Unnamed: 0'], ','.join(to_print)))    
            continue
        
    # if we get to this point, the movie was found somewhere
        
    for idx in indices:
        if years[idx] == year:
            true_idx = idx
            break

    # at this point we've found the true match of the movie
    for col in char_str_cols:
        movie_df.loc[true_idx, col] = row[col]

100%|████████████████████████████████████████████████████████████████████████████████| 855/855 [01:00<00:00, 14.17it/s]


movie_df.to_csv('edited_movie_dataset.csv', index=False)

In [35]:
with open("unmatched_movies.txt", "w", encoding="utf-8") as output:
    for movie in unmatched_movies:
        output.write(str(movie) + '\n')

##### Add manually movies

#movie_df = pd.read_csv('edited_movie_dataset.csv', low_memory=False)
#paper_df = pd.read_csv('./paper_dataset.csv')

In [36]:
real_names = {
    ' Beautiful Mind, A (2001)': 'tt0268978',
    '  Absent-Minded Professor, The (1961)': 'tt0054594',
    'Conversation(s) with Other Women (2005)': 'tt0435623',
    'Emperor’s Club, The (2002)': 'tt0283530',
    ' Hobart Shakespeareans, The (2005)': 'tt0473215',
    ' I’m Not There (2007)': 'tt0368794',
    ' Mr. Holland’s Opus (1995)': 'tt0113862',
    ' Overture, The (2004, Thailand)': 'tt0415046',
    ' Pan’s Labyrinth (2006, Mexico)': 'tt0457430',
    ' Right Now (2004, France)': 'tt0407342',
    ' Terminal, The (2004)': 'tt0362227',
    ' Vincent and Theo (1990, Netherlands)': 'tt0100873',
    ' Yes Men, The(2003)': 'tt0379593',
    ' Adventures of Robinson Crusoe, The (1954)': 'tt0044386',
    'I Am Curious (Yellow) (1967, Sweden)': 'tt0061834',
    ' Machinist, The (2004, Spain)': 'tt0361862',
    ' 265Man Without a Past, The (2002, Finland)': 'tt0311519',
    ' Mongolian Ping-Pong (2005, China)': 'tt0461804',
    ' Secret Garden, The (1993)': 'tt0108071',
    ' Star Maker, The (1995, Italy)': 'tt0114808',
    ' Darwin Awards, The(2006)': 'tt0428446',
    ' Matrix, The (1999; 2003; 2003)': ['tt0133093', 'tt0234215', 'tt0242653'],
    ' Pure Formality, A (1994, France)': 'tt0110917',
    ' Chorus, The (2004)': 'tt0372824',
    ' Wild Parrots of Telegraph Hill, The (2003)':  'tt0424565',
    'Antonia’s Line (1995, Netherlands)': 'tt0112379',
    ' Fountain, The (2006)': 'tt0414993',
    ' Karate Kid, The (1984)': 'tt0087538',
    'Lord of the Rings, The (2001; 2002; 2003)': ['tt0120737', 'tt0167261', 'tt0167260'],
    ' Celebration, The (1998, Denmark)': 'tt0154420',
    ' Constant Gardener, The (2005,  Germany/U.K.)': 'tt0387131',
    ' Diary of Anne Frank, The (1959)': 'tt0052738',
    ' Don’t Tell (2005, Italy)': 'tt0417077',
    ' Emmanuel’s Gift (2005)': 'tt0447016',
    ' Europa, Europa (1991, Germany)': 'tt0099776',
    ' Fugitive,The (1993)': 'tt0106977',
    ' Good Night, and Good Luck (2005)': 'tt0433383',
    ' Ground Truth, The (2006)': 'tt0446345',
    ' Kite Runner, The (2007)': 'tt0419887',
    ' Man Escaped, A (1956, France)': 'tt0049902',
    ' Manchurian Candidate, The (1962; 2004)': 'tt0056218',
    ' Schindler’s List (1993)': 'tt0108052',
    ' Silence of the Lambs (1991)': 'tt0102926',
    ' Sound of Music, The (1965)': 'tt0059742',
    ' 40-Year-Old Virgin, The (2005)': 'tt0405422',
    ' All the President’s Men (1976)': 'tt0074119',
    ' Captain Courageous (1937)': 'tt0028691',
    ' Civil Action, A (1998)': 'tt0120633',
    ' Contender,The (2000)': 'tt0208874', 
    ' Dirty, Filthy Love (2004, U.K.)': 'tt0411291',
    ' Erin Brokovich (2000)': 'tt0195685',
    ' Few Good Men, A (1992)': 'tt0104257',
    ' Legend of Bagger Vance, The (2000)': 'tt0146984',
    ' Lives of Others, The (2006)': 'tt0405094',
    ' Papillion (1973)': 'tt0070511',
    ' Piano, The (1993, Australia)': 'tt0107822',
    ' 267Prime (2005)': 'tt0387514',
    ' Right Stuff, The (1983)': 'tt0086197',
    ' Verdict, The (1982)': 'tt0084855',
    ' Willy Wonka and the Chocolate Factory (1971)': 'tt0067992',
    '400 Blows, The (1959, France)': 'tt0053198',
    ' Long Life, Happiness and Prosperity (2002, Canada)': 'tt0323103',
    ' Man Who Wasn’t There, The (2001)': 'tt0243133',
    ' One Flew Over the Cuckoo’s Nest (1975)': 'tt0073486',
    ' Singin’ in the Rain (1952)': 'tt0045152',
    ' What’s Love Got to Do with It? (1993)': 'tt0108551',
    ' World’s Fastest Indian, The (2005, New Zealand)': 'tt0412080',
    ' Ali Zoua: Prince of the Streets (2000, Morocco)': 'tt0260688',
    ' Bicycle Thief, The (1948, France)': 'tt0040522',
    ' Born into Brothels (2004, India)': 'tt0388789',
    ' City of Lost Children, The (1995, France)': 'tt0112682',
    ' Color Purple, The (1985)': 'tt0088939',
    ' Da Vinci Code, The (2006)': 'tt0382625',
    ' Diving Bell and the Butterfly, The (2007, France/U.S.)': 'tt0401383',
    ' East-West (1999, France)': 'tt0181530',
    ' Endurance, The (2000, Germany/U.S.)': 'tt0264578',
    ' Grapes of Wrath, The (1940)': 'tt0032551',
    ' Great Debaters, The (2007)': 'tt0427309',
    ' Harry Potter (2001; 2002; 2004; 2005; 2007)': ['tt0241527', 'tt0295297', 'tt0304141', 'tt0330373', 'tt0373889', 'tt0417741', 'tt0926084', 'tt1201607'],
    ' Hours, The(2002)': 'tt0274558',
    ' Island on Bird Street, The (2000)': 'tt0119389',
    ' Italian, The (2005, Russia)': 'tt0450450',
    '268Kill Bill: Vol. 1 and Vol. 2 (2003; 2004)': ['tt0266697', 'tt0378194'],
    ' Maria, Full of Grace (2004, Colum-bia/U.S.)': 'tt0390221',
    ' Mighty Heart, A (2007)': 'tt0829459',
    ' Old Well (1986, China)': 'tt0091377',
    ' Pianist, The (2002, France/Germany/U.K./Poland)': 'tt0253474',
    ' Pride of the Yankees, The (1942)': 'tt0035211',
    ' Pursuit of Happyness, The (2006)': 'tt0454921',
    '  Return, The (2003, Russia)': 'tt0376968',
    ' Secret of Dr. Kildare, The (1939)': 'tt0031909',
    ' Spirit of St. Louis, The (1957)': 'tt0051003',
    ' Story of Qiu Ju, The (1992, China/Hong Kong)': 'tt0105197',
    ' Affair to Remember, An (1957)': 'tt0050105',
    ' Breakfast at Tiffany’s (1961)': 'tt0054698',
    ' Bridges of Madison County, The (1995)': 'tt0112579',
    ' Dr. Zhivago (1965)': 'tt0059113',
    ' English Patient, The (1996)': 'tt0116209',
    ' Fisher King, The (1991)': 'tt0101889',
    ' Green Mile, The (1999)': 'tt0120689',
    ' 269Happy Endings (2005)': 'tt0361693',
    ' Hired Heart, The (1997)': 'tt0128251',
    ' Holiday, The (2006)': 'tt0457939',
    ' Illusionist, The (2006)': 'tt0443543',
    ' Kid, The (1921)': 'tt0012349',
    ' Lake House, The (2006)': 'tt0410297',
    ' Last Kiss, The (2006)': 'tt0434139',
    ' Like Water for Chocolate (1992, Mexico)': 'tt0103994',
    ' Monsters Ball (2001)': 'tt0285742',
    ' Namesake, The (2006, India/U.S.)': 'tt0433416',
    ' New World, The (2005)': 'tt0402399',
    ' Notebook, The (2004)': 'tt0332280',
    ' Officer and a Gentleman, An (1982)': 'tt0084434',
    ' OH in Ohio, The (2006)': 'tt0422861',
    ' Paris, I Love You (2006, France)': 'tt0401711',
    ' Perfect World, A (1993)': 'tt0107808',
    ' Phantom of the Opera, The (2004)': 'tt0293508',
    ' Postman, The (Il Postino, 1994, France)': 'tt0110877',
    ' Princess Bride, The (1987)': 'tt0093779',
    ' River Runs Through It, A (1992)': 'tt0105265',
    ' Road Home, The (1999, China)': 'tt0235060',
    ' Short Film About Love, A (1988, Poland)': 'tt0095467',
    ' Something’s Gotta Give (2003)': 'tt0337741',
    ' Sophie’s Choice (1982)': 'tt0084707',
    ' Story of Us, The (1999)': 'tt0160916',
    ' Taste of Others, The (2000, France)': 'tt0216787',
    ' Thelma and Louise (1991)': 'tt0103074',
    ' Tiger and the Snow, The (2005, Italy)': 'tt0419198',
    ' When Harry Met Sally 1989)': 'tt0098635',
    ' Amelie (2001, France)': 'tt0211915',
    ' Babe; Babe: Pig in the City (1995; 1998)': 'tt0120595',
    ' Babette’s Feast (1987, Denmark)': 'tt0092603',
    ' Bagdad Café (1987, West Germany/U.S.)': 'tt0095801',
    ' Bear, The (1988, France)': 'tt0095800',
    ' Cherry Orchard, The (1999, Greece/Cyprus/France)': 'tt0144134',
    ' Christmas Carol, A (1984)': 'tt0087056',
    ' Cider House Rules (1999)': 'tt0124315',
    ' Cup, The (1999, Bhutan)': 'tt0201840',
    ' Family Man, The (2000)': 'tt0218967',
    ' House of Sand & Fog (2003)': 'tt0315983',
    ' Savages, The (2007)': 'tt0775529',
    'African Queen, The (1951)': 'tt0043265',
    ' Coal Miner’s Daughter (1980)': 'tt0080549',
    ' Don’t Come Knocking (2005)': 'tt0377800',
    ' Five Senses, The (1999)': 'tt0168794',
    ' Great New Wonderful, The (2005)': 'tt0402230',
    ' History of Violence, A (2005)': 'tt0399146',
    ' Killing Fields,The (1984)': 'tt0087553',
    ' Bad News Bears, The (1976; 2005)': 'tt0074174',
    ' Batteries Not Included (1987)': 'tt0092494',
    ' Best Years of Our Lives, The (1946)': 'tt0036868',
    ' 271Divine Secrets of the Ya-ya Sisterhood, The (2002)': 'tt0279778',
    ' Game of Their Lives, The (2005)': 'tt0354595',
    ' League of Their Own, A(1992)': 'tt0104694',
    ' Longest Yard, The (2005)': 'tt0398165',
    ' Mighty Ducks, The (1992)': 'tt0104868',
    ' Mighty Wind, A (2003)': 'tt0310281',
    ' People Say I’m Crazy (2003)': 'tt0379926',
    ' Prairie Home Companion, A (2006)': 'tt0420087',
    ' Rookie, The (2002)': 'tt0265662',
    ' Sea Inside, The (2004, Spain)': 'tt0369702',
    ' Shipping News, The (2001)': 'tt0120824',
    ' Shrek; Shrek 2; Shrek the Third (2001; 2004; 2007)': ['tt0126029', 'tt0298148', 'tt0413267'],
    ' Sisterhood of the Traveling Pants, The (2005)': 'tt1018785',
    ' Station Agent, The (2003)': 'tt0340377',
    ' Widow of Saint Pierre, The (2000, France)': 'tt0191636',
    ' Adventures of Robin Hood, The (1938)': 'tt0029843',
    ' Gentleman’s Agreement (1947)': 'tt0039416',
    ' Girl in the Café, The (2005, U.K./U.S.)': 'tt0443518',
    ' Guess Who’s Coming to Diner (1967)': 'tt0061735',
    ' Last Wave, The (1977, Australia)': 'tt0076299',
    ' Motorcycle Diaries (2004, Argentina)': 'tt0318462',
    ' Sullivan’s Travels (1941)': 'tt0034240',
    ' X-Men, X2, X-Men: The Last Stand (2000; 2003; 2006)': ['tt0120903', 'tt0290334', 'tt0376994'],
    ' Inconvenient Truth, An (2006)': 'tt0497116',
    ' Jimmy Carter: Man From Plains (2007)': 'tt0913958',
    ' Last King of Scotland, The (2006)': 'tt0455590',
    ' Lion King, The (1994)': 'tt0110357',
    ' Man for All Seasons, A (1966)': 'tt0060665',
    ' Prize Winner of Defiance Ohio (2005)': 'tt0406158',
    ' Sixth Sense, The (1999)': 'tt0167404',
    ' Twelve O’Clock High (1949)': 'tt0041996',
    ' Angela’s Ashes (1999)': 'tt0145653',
    ' City of No Limits, The (2002, Spain)': 'tt0311072',
    ' Crossing Guard, The (1995)': 'tt0112744',
    ' No Man’s Land (2002, Bosnia-Herzegovina)': 'tt0283509',
    ' O Brother, Where Art Thou (2000)': 'tt0190590',
    ' Squid and the Whale, The (2005)': 'tt0367089',
    ' Straight Story, The (1999)': 'tt0166896',
    ' Thousand Acres, A (1997)': 'tt0120323',
    ' Unfinished Life, An (2005)': 'tt0350261',
    ' Upside of Anger, The (2005)': 'tt0365885',
    ' Fourth Wise Man, The (1985)': 'tt0089165',
    ' Keys of the Kingdom, The (1944)': 'tt0036983',
    ' 273My Super Ex-Girlfriend (2006)': 'tt0465624',
    ' Passion of the Christ, The (2004)': 'tt0335345',
    ' Ringer, The (2005)': 'tt0267891',
    ' Truman Show, The (1998)': 'tt0120382',
    ' Day the Earth Stood Still,The (1951)': 'tt0970416',
    ' Queen, The (2006, U.K.)': 'tt0436697',
    ' Road Home,The (1999, China)': 'tt0235060',
    ' Shawshank Redemption, The (1994)': 'tt0111161',
    ' Story of the Weeping Camel, The (2004, Mongolia)': 'tt0373861',
    ' Bourne Identity; Bourne Supremacy; Bourne Ultimatum, The (2002; 2004; 2007)': ['tt0372183', 'tt0440963', 'tt0258463'],
    ' Bridge on the River Kwai, The (1957)': 'tt0050212',
    ' Brothers, The (2001)': 'tt0250274',
    ' Maltese Falcon, The (1941)': 'tt0022111',
    ' Paper Chase, The (1973)': 'tt0070509',
    ' Snow Walker, The (2003, Canada)': 'tt0337721',
    'Spring, Summer, Fall, Winter…and Spring (2003, South Korea)': 'tt0374546',
    ' Syrian Bride,The (2004, Israel/France)': 'tt0423310',
    ' Warrior, The (2001)': 'tt0295682',
    ' Woodsman, The (2004)': 'tt0361127',
    ' America’s Heart and Soul (2004)': 'tt0381006',
    ' Celestine Prophecy (2006)': 'tt0398842',
    ' Color of Paradise, The (1999, Iran)': 'tt0191043',
    ' It’s a Wonderful Life (1946)': 'tt0038650',
    ' Pillow Book, The (1997, France/U.K.)': 'tt0114134',
    ' Prestige, The (2006)': 'tt0482571',
    ' Scent of Green Papaya, The (1993,  Vietnam)': 'tt0107617',
    ' Vertical Ray of the Sun, The (2000,  Vietnam)': 'tt0224578',
    ' Wind Will Carry Us, The (1999, Iran)': 'tt0209463',
    ' Doctor, The (1991)': 'tt0101746',
    ' For One More Day (2007)': 'tt1023504',
    ' Good Year, A (2006)': 'tt0401445',
    ' Last Samurai, The (2003)': 'tt0325710',
    ' Rory O’Shea Was Here (2004)': 'tt0417791',
    ' Ultimate Gift, The (2006)': 'tt0482629',
    ' Willow Tree, The (2005, Iran)': 'tt0415607',
    ' Black Stallion, The (1979)': 'tt0078872',
    ' Blue Butterfly (2004)': 'tt0313300',
    ' Cooler, The (2003)': 'tt0318374',
    ' E.T.: The Extra-Terrestrial (1982)': 'tt0083866',
    ' Hustle and Flow (2005)': 'tt0410097',
    ' Maldonado Miracle, The (2003)': 'tt0328097',
    ' Miracle Worker, The (1962)': 'tt0056241',
    ' Nativity Story, The (2006)': 'tt0762121',
    ' Perez Family, The (1995)': 'tt0114113',
    ' Raisin in the Sun, A (1961)': 'tt0055353',
    ' Very Long Engagement, A (2004, France)': 'tt0344510',
    ' Wizard of Oz, The (1939)': 'tt0032138',
    ' Laughing Club of India, The (1999, India)': 'tt0283448',
    ' Road to Welville, The (1994)': 'tt0111001',
    '11’09”01 – September 11 (2002)': 'tt0328802',
    ' 13 Conversations About One Thing (2001)': 'tt0268690',
    ' Apostle, The (1987)': 'tt0118632',
    ' Astronaut Farmer, The (2006)': 'tt0469263',
    ' Believer, The (2001)': 'tt0247199',
    ' Bible, The (1966)': 'tt0060164',
    ' Big Kahuna, The (1999)': 'tt0189584',
    ' Book of Life, The (1998)': 'tt0167059',
    ' Bucket List, The (2007)': 'tt0825232',
    ' Chronicles of Narnia: The Lion, the Witch, and the Wardrobe, The (2005)': 'tt0363771',
    ' Crime of Father Amaro, The (2002, Mex-ico)': 'tt0313196',
    ' Day Christ Died, The (1980)': 'tt0080595',
    'Emmanuel’s Gift (2005)': 'tt0447016',
    ' From the Manger to the Cross (1912)': 'tt0002199',
    ' God Is Great and I Am Not (2001, France)': 'tt0287986',
    ' Golden Compass, The (2007)': 'tt0385752',
    ' Gospel According to Saint Matthew, The (1964, Italy)': 'tt0058715',
    ' Gospel of John, The (2003)': 'tt0377992',
    ' Greatest Story Ever Told, The (1965)': 'tt0059245',
    ' Heaven’s Bookstore (2004, Japan)': 'tt0423360',
    ' Hitchhiker’s Guide to the Galaxy, The (2005)': 'tt0371724',
    ' Incredibles, The (2004)': 'tt0317705',
    ' James’ Journey to Jerusalem (2003, Is-rael)': 'tt0305178',
    ' King of Kings, The (1961)': 'tt0055047',
    ' Last Temptation of Christ, The (1988)': 'tt0095497',
    ' Miracle Maker, The (2000)': 'tt0208298',
    ' Monty Python’s Life of Brian (1979)': 'tt0079470',
    ' Monty Python’s the Meaning of Life (1983, U.K.)': 'tt0472471',
    ' Princess and the Warrior, The (2000, Germany)': 'tt0203632',
    ' Razor’s Edge, The (1984)': 'tt0087980',
    ' Secret of Roan Inish (1994, U.S./Ireland)': 'tt0111112',
    ' Seventh Seal, The (1957, Sweden)': 'tt0050976',
    ' 277Sliding Doors (1998)': 'tt0120148',
    ' Tao of Steve, The (2000)': 'tt0234853',
    ' Ten Commandments, The (1956)': 'tt0049833',
    ' Therese: Story of St. Therese of Liseaux (2004)': 'tt0286218',
    ' War Within, The (2005)': 'tt0414344',
    ' Weather Man, The (2005)': 'tt0384680',
    ' Y Tu Mama Tambien (2001, Mexico)': 'tt0245574',
}

#### There are 6 movies that don't exist in the IMDB database (mainly videos)

In [37]:
char_str_cols = paper_df.columns.to_list()[1:-1]
paper_movies = []

for movie in paper_df['Unnamed: 0']:
    
    paper_movies.append(movie.strip())

for movie_name in tqdm(real_names.keys()):
    
    if type(real_names[movie_name]) != list: 
    
        relevant_row = paper_df.loc[paper_movies.index(movie_name.strip()), :] 
        idx = np.where(movie_df['tconst'] == real_names[movie_name])
        
        for col in char_str_cols:
            movie_df.loc[idx[0], col] = int(relevant_row[col])
            
    else: 
        
        for mov in real_names[movie_name]:
            
            relevant_row = paper_df.loc[paper_movies.index(movie_name.strip()), :]
            idx = np.where(movie_df['tconst'] == mov)
        
            for col in char_str_cols:
                movie_df.loc[idx[0], col] = int(relevant_row[col])
    

100%|████████████████████████████████████████████████████████████████████████████████| 282/282 [00:20<00:00, 14.05it/s]


#### Create column number of meaningful votes

In [38]:
num_meaning_votes = movie_df['Meaningfulness'].apply(lambda x: 1 if x > 0.0 else 0)
movie_df.insert(loc=9, column='numMeaningVotes', value=num_meaning_votes)

In [39]:
movie_df.rename(columns = {'tconst': 'movieId',
                           'numVotes': 'numEnjoyVotes',
                           'newAverageRating': 'averageEnjoyRating',
                           'Meaningfulness': 'averageMeaningRating'
                          }, inplace = True)

movie_df = movie_df.drop(columns=['Unnamed: 0'])

### Keep only the movies with character strengths

In [40]:
has_ch_strengths = movie_df['averageMeaningRating'] > 0.0
ch_movie_df = movie_df[has_ch_strengths]

### Get the movie posters

In [41]:
API_KEY = 'f3090026c1cc5b7fbf9fc9b8d619fe57'

def get_poster_url(key, movie_id):

    CONFIG_PATTERN = 'http://api.themoviedb.org/3/configuration?api_key={key}'

    url = CONFIG_PATTERN.format(key=key)
    r = requests.get(url)
    config = r.json()

    IMG_PATTERN = 'http://api.themoviedb.org/3/movie/{imdbid}/images?api_key={key}'

    r = requests.get(IMG_PATTERN.format(key=key,imdbid=movie_id))
    api_response = r.json()

    try:
        api_response['id']
        # sizes are (as stored): ['w92', 'w154', 'w185', 'w342', 'w500', 'w780', 'original']
        url = config['images']['secure_base_url'] + config['images']['poster_sizes'][0] + api_response['posters'][0]['file_path']
        
    except:
        #print(f"The poster was not found for movie {movie_id}")
        url = 'Unknown'
        
    return url

In [42]:
poster_urls = []

for _, row in tqdm(ch_movie_df.iterrows(), total=ch_movie_df.shape[0]):
    poster_urls.append(get_poster_url(API_KEY, row['movieId']))
    
ch_movie_df.insert(loc=6, column='posterUrl', value=poster_urls)

100%|████████████████████████████████████████████████████████████████████████████████| 670/670 [02:08<00:00,  5.23it/s]


### Separate the database in two tables: one containing info and the other ratings

In [44]:
movie_info_df = ch_movie_df.iloc[:, :7]
movie_ratings_df = ch_movie_df.iloc[:, 8:]
movie_ratings_df.insert(loc=0, column='movieId', value=ch_movie_df['movieId'])

In [45]:
movie_info_df.to_csv('movie_info.csv', index=False)
movie_ratings_df.to_csv('movie_ratings.csv', index=False)